In [10]:
# Setup the Dash version for Jupyter
from dash import Dash, dcc, html, dash_table, Input, Output, State, callback_context
import dash_bootstrap_components as dbc

# Configure the necessary Python module imports
import dash_leaflet as dl
import pandas as pd
import plotly.express as px

# Import CRUD module
from animal_shelter import AnimalShelter

# New imports for the added features
import io
import base64

# Define constants for repeated values
BACKGROUND_COLOR = '#222'
FOREGROUND_COLOR = '#333'
TEXT_COLOR = 'white'
ACCENT_COLOR = '#00CED1'

###########################
# Data Manipulation / Model
###########################
# App instantiation
app = Dash('GraziosoSalvareDashboard', suppress_callback_exceptions=True, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Login screen layout
auth_layout = html.Div([
    html.H2('Login to Grazioso Salvare Dashboard', style={'textAlign': 'center', 'color': '#00CED1'}),
    html.Div([
        html.Label('Username:'),
        dcc.Input(id='login-username', type='text', placeholder='Enter your username'),
        html.Br(),
        html.Label('Password:'),
        dcc.Input(id='login-password', type='password', placeholder='Enter your password'),
        html.Br(),
        html.Button('Submit', id='login-button', n_clicks=0),
    ], style={'padding': '20px', 'backgroundColor': '#333', 'color': 'white', 'width': '300px', 'margin': '0 auto'}),
    html.Div(id='login-error', style={'textAlign': 'center', 'color': 'red'})
], style={'backgroundColor': '#222', 'color': 'white', 'height': '100vh', 'display': 'flex', 'flexDirection': 'column', 'justifyContent': 'center', 'alignItems': 'center'})

# CRUD layout
crud_layout = html.Div([
    dbc.Modal([
        dbc.ModalHeader("Add/Edit Animal Record"),
        dbc.ModalBody([
            dbc.Input(id="animal-id-input", placeholder="Animal ID", type="text"),
            dbc.Input(id="animal-type-input", placeholder="Animal Type", type="text"),
            dbc.Input(id="breed-input", placeholder="Breed", type="text"),
            dbc.Input(id="color-input", placeholder="Color", type="text"),
            dbc.Input(id="age-input", placeholder="Age Upon Outcome", type="text"),
            dbc.Input(id="outcome-type-input", placeholder="Outcome Type", type="text"),
        ]),
        dbc.ModalFooter([
            dbc.Button("Save", id="save-record-button", className="ml-auto"),
            dbc.Button("Close", id="close-modal-button", className="ml-2"),
        ]),
    ], id="crud-modal"),
    dbc.Button("Add New Record", id="open-modal-button", color="primary", className="mb-3"),
    dbc.Button("Delete Selected Record", id="delete-record-button", color="danger", className="mb-3 ml-2"),
])

# Dashboard layout
main_layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    
    html.Div([
        html.Center(html.B(html.H1('Grazioso Salvare Dashboard', 
                                   style={'color': '#00CED1', 'fontFamily': 'Arial'}))),
        html.A(
            href='https://www.snhu.edu',  # Link to the desired URL
            target='_blank',  # Opens the link in a new tab
            children=[
                html.Img(src='/assets/grazioso_logo.png', style={
                    'position': 'fixed', 
                    'top': '10px', 
                    'right': '10px', 
                    'width': '100px', 
                    'zIndex': '1000'
                })
            ]
        )
    ], style={'position': 'relative'}),
    html.Hr(),

    # Search fields for filtering
    html.Div([
        html.Label('Search:'),
        dcc.Input(
            id='combined-search', 
            type='text', 
            placeholder='Enter animal ID, breed, animal type, or outcome type', 
            style={'width': '325px'}
        ),
        html.Br(),
        html.Br(),
        html.Label('Rescue Type:'),
        dcc.RadioItems(
            id='rescue-type-radio',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}, 
            style={'color': 'white', 'display': 'flex', 'flex-wrap': 'wrap'}
        ),
    ], style={'padding': '20px', 'backgroundColor': '#333', 'color': 'white'}),
    
    html.Hr(),
    
    # CRUD layout
    crud_layout,
    
    # DataTable with added style for better visibility
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": "Animal ID", "id": "animal_id"},  
            {"name": "Animal Type", "id": "animal_type"},
            {"name": "Breed", "id": "breed"},
            {"name": "Color", "id": "color"},
            {"name": "Age Upon Outcome", "id": "age_upon_outcome"},
            {"name": "Outcome Type", "id": "outcome_type"}
        ],
        data=[],  
        page_size=10,  
        row_selectable="single",  
        sort_action="native",  

        # Ensure both horizontal and vertical scrolling are enabled
        style_table={
            'overflowX': 'auto',  # Ensures scrolling
            'overflowY': 'auto',
            'maxHeight': '300px', 
            'minWidth': '1000px'  # Ensures a minimum width for proper data display
        },

        # Ensure that cell content doesn't get cut off
        style_cell={
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'textAlign': 'left'
        },

        # Styling for data cells
        style_data={'backgroundColor': '#333', 'color': 'white'},

        # Alternating row colors and selection feedback
        style_data_conditional=[
            {'if': {'row_index': 'odd'}, 'backgroundColor': '#444'},
            {'if': {'state': 'selected'}, 'backgroundColor': '#FFD700', 'color': 'black'}
        ],

        # Header styling
        style_header={'backgroundColor': '#555', 'color': 'white', 'fontWeight': 'bold'}
    ),

    html.Br(),
    html.Hr(),
    
    # Export Data Button
    html.Button("Export Data", id="export-button", n_clicks=0, style={'margin': '10px'}),
    dcc.Download(id="download-dataframe-csv"),
    
    # Map and Pie Chart side by side
    html.Div([
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'height': '500px', 'flex': '1', 'display': 'inline-block', 'verticalAlign': 'top', 'marginRight': '1%'}
        ),
        
        html.Div([
            dcc.Graph(
                id='breed-pie-chart',
                style={'height': '500px', 'flex': '1', 'display': 'inline-block', 'verticalAlign': 'top'}
            )
        ], style={'flex': '1'})
    ], style={'display': 'flex', 'flexDirection': 'row', 'width': '100%', 'gap': '0px'}),
    
    # Footer with author name
    html.Div([
        html.H6('By Alexander Wagner', style={'color': 'white', 'textAlign': 'center', 'marginTop': '20px'})
    ])
], style={'backgroundColor': '#222', 'color': 'white', 'padding': '20px'})

#############################################
# Interaction Between Components / Controller
#############################################

# Main app layout callback to switch between authentication and main layout
@app.callback(
    Output('app-layout', 'children'),
    Output('login-error', 'children'),
    [Input('login-button', 'n_clicks')],
    [State('login-username', 'value'), State('login-password', 'value')]
)
def authenticate_user(n_clicks, username, password):
    if n_clicks > 0:
        try:
            # Always use the latest input values, or fallback to defaults if empty
            username = username or 'aacuser'
            password = password or 'SNHU1234'
            
            global shelter
            shelter = AnimalShelter(username, password)

            # Test MongoDB connection by attempting a read operation
            test_df = pd.DataFrame.from_records(shelter.read({}))
            if not test_df.empty:
                return main_layout, ''
            else:
                return auth_layout, 'No data available. Please check your credentials or try again later.'
        except Exception:
            # Clear the login input fields by returning an error message
            return auth_layout, 'Login failed. Please check your username and password.'
    return auth_layout, ''

# Callback to filter the data table based on search inputs and rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('combined-search', 'value'), Input('rescue-type-radio', 'value')]
)
def filter_table(combined_search, rescue_type):
    # Retrieve all data from MongoDB
    filtered_df = pd.DataFrame.from_records(shelter.read({}))
    
    # Drop the '_id' column if it exists to clean up the DataFrame
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    
    # Apply search filtering if search input is provided
    if combined_search:
        filtered_df = filtered_df[filtered_df.apply(
            lambda row: (
                combined_search.lower() in str(row['animal_id']).lower() or
                combined_search.lower() in str(row['breed']).lower() or
                combined_search.lower() in str(row['animal_type']).lower() or
                combined_search.lower() in str(row['outcome_type']).lower()
            ), axis=1
        )]
    
    # Apply rescue type filtering if a type is selected
    if rescue_type and rescue_type != 'Reset':
        if rescue_type == 'Water Rescue':
            # Water Rescue breed filter
            filtered_df = filtered_df[filtered_df['breed'].str.contains('Labrador Retriever Mix|Chesapeake Bay Retriever|Newfoundland', case=False, na=False)]
        elif rescue_type == 'Mountain or Wilderness Rescue':
            # Mountain or Wilderness breed filter
            filtered_df = filtered_df[filtered_df['breed'].str.contains('German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler', case=False, na=False)]
        elif rescue_type == 'Disaster or Individual Tracking':
            # Disaster or Individual Tracking breed filter
            filtered_df = filtered_df[filtered_df['breed'].str.contains('Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler', case=False, na=False)]
    
    # Return the filtered data as a list of dictionaries
    return filtered_df.to_dict('records')

# Callback to update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if not viewData or selected_rows is None or len(selected_rows) == 0:
        return [dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[dl.TileLayer(id="base-layer-id")])]

    dff = pd.DataFrame.from_dict(viewData)
    row = selected_rows[0] if selected_rows else None

    if row is None:
        return [dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[dl.TileLayer(id="base-layer-id")])]

    # Get the necessary data for the map update using correct column names
    animal_name = dff.iloc[row].get('name', 'Unknown')
    animal_id = dff.iloc[row].get('animal_id', 'Unknown')
    lat = dff.iloc[row].get('location_lat', 30.75)
    lon = dff.iloc[row].get('location_long', -97.48)

    try:
        lat = float(lat)
        lon = float(lon)
    except (ValueError, TypeError):
        lat, lon = 30.75, -97.48

    return [
        dl.Map(
            id="map",  # Ensure a unique id is used
            style={'width': '100%', 'height': '500px'},
            center=[lat, lon],  # Force center to be recalculated on selection
            zoom=13,  
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(dff.iloc[row].get('breed', 'Unknown')),
                        dl.Popup([
                            html.H1(f"Animal Name: {animal_name}"),
                            html.P(f"Animal ID: {animal_id}")
                        ])
                    ]
                )
            ]
        )
    ]

# Callback to update the pie chart based on the filtered data
@app.callback(
    Output('breed-pie-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_pie_chart(data):
    if data is None or len(data) == 0:
        return {}

    dff = pd.DataFrame(data)
    breed_counts = dff['breed'].value_counts()
    top_breeds = breed_counts.nlargest(10).index
    dff['breed_grouped'] = dff['breed'].apply(lambda x: x if x in top_breeds else 'Other')
    
    fig = px.pie(dff, names='breed_grouped', title='Top 10 Breed Distribution', hole=0.3)
    fig.update_layout(
        title_font_size=20,
        legend_title_text='Breed'
    )
    
    return fig

# Callback for Export Data Feature
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    State("datatable-id", "data"),
    prevent_initial_call=True,
)
def export_data(n_clicks, data):
    if n_clicks > 0:
        df = pd.DataFrame(data)
        return dcc.send_data_frame(df.to_csv, "exported_data.csv")

# Callbacks for CRUD operations

@app.callback(
    Output("crud-modal", "is_open"),
    [Input("open-modal-button", "n_clicks"), Input("close-modal-button", "n_clicks"), Input("save-record-button", "n_clicks")],
    [State("crud-modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open

@app.callback(
    Output("datatable-id", "data", allow_duplicate=True),
    [Input("save-record-button", "n_clicks"), Input("delete-record-button", "n_clicks")],
    [State("animal-id-input", "value"),
     State("animal-type-input", "value"),
     State("breed-input", "value"),
     State("color-input", "value"),
     State("age-input", "value"),
     State("outcome-type-input", "value"),
     State("datatable-id", "selected_rows"),
     State("datatable-id", "data")],
    prevent_initial_call=True
)
def handle_crud_operations(save_clicks, delete_clicks, animal_id, animal_type, breed, color, age, outcome_type, selected_rows, current_data):
    ctx = callback_context
    if not ctx.triggered:
        return current_data
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if button_id == "save-record-button" and save_clicks:
        new_record = {
            "animal_id": animal_id,
            "animal_type": animal_type,
            "breed": breed,
            "color": color,
            "age_upon_outcome": age,
            "outcome_type": outcome_type
        }
        shelter.create(new_record)
    elif button_id == "delete-record-button" and delete_clicks and selected_rows:
        record_to_delete = current_data[selected_rows[0]]
        shelter.delete({"animal_id": record_to_delete["animal_id"]})
    
    # Refresh the data after CRUD operation
    updated_data = pd.DataFrame.from_records(shelter.read({}))
    if '_id' in updated_data.columns:
        updated_data.drop(columns=['_id'], inplace=True)
    return updated_data.to_dict('records')

# Run the app
app.layout = html.Div(id='app-layout', children=auth_layout)
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)